In [1]:
#Import relevant libraries
import io
import itertools
import sklearn.metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorboard.plugins.hparams import api as hp

In [2]:
#Dataset encoding explaination.
#image labelling - 0=glasses/sunglasses. 1= trousers/jeans. 3= shoes

In [3]:
#Import the datasets and preprocess

data_train = np.load(r'Full Dataset/Trousers & Jeans - All - Train.npz')
data_validation = np.load(r'Full Dataset/Trousers & Jeans - All - Validation.npz')
data_test = np.load(r'Full Dataset/Trousers & Jeans - All - Test.npz')


#t extract the arrays from dataset into input(images) and target(labels)
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_validation['images']
labels_val = data_validation['labels']


images_test = data_test['images']
labels_test = data_test['labels']

#Pixel-wise normalization of the training, validation and testing data
images_train = images_train/255.0
images_val  = images_val/255.0
images_test = images_test/255.0







In [4]:
#Define the hyperparameters

BATCH_SIZE = 64
EPOCHS = 15

#Define the hyperparamets to tune and the variations we want to test.
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filter_num', hp.Discrete([32, 64, 96, 128]))


METRIC_ACCURACY = 'accuracy'

#Log the hyperparameter with the file writer
with tf.summary.create_file_writer('Logs/ModelG 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM], 
        metrics= [hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )



In [5]:
def train_model(hparams,session_num):
    
    
#Create and train the model


    
    model = Sequential([
                Conv2D(32, 3, activation= 'relu', input_shape=(120,90,3)),
                MaxPooling2D(pool_size=(2,2)),
                Conv2D(64, 3, activation= 'relu'),
                MaxPooling2D(pool_size=(2,2)),
                Flatten(),
                Dense(64, activation = 'relu'),
                
                Dense(4)
    ])


#describe the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Compile the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    log_dir = f"Logs/ModelG 1/fit/run-{session_num}"

    
        
        
        
#Plot_to_image

    def plot_confusion_matrix(cm, class_names):
 
        figure = plt.figure(figsize=(8, 8))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Compute the labels from the normalized confusion matrix.
        labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    



    def plot_to_image(figure):
       
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image


        
    # log the confusion matrix as a heatmap
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')


    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        val_pred_raw = model.predict(images_val)
        val_pred = np.argmax(val_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = confusion_matrix(labels_val, val_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=["Trouser Male", "Trouse Female", "Jeans Male", "Jeans Female"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion_matrix", cm_image, step=epoch)

    # Define the per-epoch callback.
    cm_callback = LambdaCallback(on_epoch_end=log_confusion_matrix)


    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


    early_stopping = EarlyStopping(
        monitor= 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0,
        restore_best_weights = True
    )


#Train the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val, labels_val),
        verbose =2
    )

    _,accuracy = model.evaluate(images_val, labels_val)

    model.save(f"saved_model/ModelG 1/fit/run-{session_num}")


    return accuracy



In [6]:
#Log the hyperparameter to the TensorBoard



def run(log_dir, hparams,session_num):
    hyperparameter_writer = tf.summary.create_file_writer(log_dir)
    
    
    with hyperparameter_writer.as_default():
        hp.hparams(hparams) #record the values used in this trial
        accuracy = train_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        
        
        
        
        
#Train the model with the different hyperparameters
session_num = 1
for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        hparams ={
        HP_FILTER_SIZE: filter_size,
        HP_FILTER_NUM : filter_num
        }
        
        run_name = "run-%d" % session_num
        print('---Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/ModelG 1/hparam_tuning/' + run_name, hparams, session_num)
        
        session_num += 1
       
        

       
        
        
        








---Starting trial: run-1
{'filter_size': 3, 'filter_num': 32}
Epoch 1/15
16/16 [==============================] - 1s 36ms/step
63/63 - 16s - loss: 1.2974 - accuracy: 0.4568 - val_loss: 1.0128 - val_accuracy: 0.6000 - 16s/epoch - 262ms/step
Epoch 2/15
16/16 [==============================] - 1s 32ms/step
63/63 - 15s - loss: 0.8694 - accuracy: 0.6503 - val_loss: 0.9055 - val_accuracy: 0.6180 - 15s/epoch - 232ms/step
Epoch 3/15
16/16 [==============================] - 1s 34ms/step
63/63 - 15s - loss: 0.7734 - accuracy: 0.6886 - val_loss: 0.9988 - val_accuracy: 0.6100 - 15s/epoch - 240ms/step
Epoch 4/15
16/16 [==============================] - 1s 34ms/step
63/63 - 16s - loss: 0.6834 - accuracy: 0.7258 - val_loss: 0.7993 - val_accuracy: 0.7020 - 16s/epoch - 253ms/step
Epoch 5/15
16/16 [==============================] - 1s 34ms/step
63/63 - 16s - loss: 0.6303 - accuracy: 0.7475 - val_loss: 0.7442 - val_accuracy: 0.7120 - 16s/epoch - 252ms/step
Epoch 6/15
16/16 [==============================

INFO:tensorflow:Assets written to: saved_model/ModelG 1/fit/run-1\assets


---Starting trial: run-2
{'filter_size': 3, 'filter_num': 64}
Epoch 1/15
16/16 [==============================] - 1s 39ms/step
63/63 - 21s - loss: 1.2376 - accuracy: 0.4880 - val_loss: 0.9874 - val_accuracy: 0.5860 - 21s/epoch - 338ms/step
Epoch 2/15
16/16 [==============================] - 1s 37ms/step
63/63 - 18s - loss: 0.8459 - accuracy: 0.6528 - val_loss: 0.8891 - val_accuracy: 0.6380 - 18s/epoch - 281ms/step
Epoch 3/15
16/16 [==============================] - 1s 38ms/step
63/63 - 17s - loss: 0.7316 - accuracy: 0.7028 - val_loss: 0.8616 - val_accuracy: 0.6440 - 17s/epoch - 264ms/step
Epoch 4/15
16/16 [==============================] - 1s 37ms/step
63/63 - 17s - loss: 0.6689 - accuracy: 0.7328 - val_loss: 0.8434 - val_accuracy: 0.6940 - 17s/epoch - 267ms/step
Epoch 5/15
16/16 [==============================] - 1s 41ms/step
63/63 - 17s - loss: 0.6011 - accuracy: 0.7675 - val_loss: 0.7608 - val_accuracy: 0.7260 - 17s/epoch - 270ms/step
Epoch 6/15
16/16 [==============================

INFO:tensorflow:Assets written to: saved_model/ModelG 1/fit/run-2\assets


---Starting trial: run-3
{'filter_size': 3, 'filter_num': 96}
Epoch 1/15
16/16 [==============================] - 1s 46ms/step
63/63 - 23s - loss: 1.2026 - accuracy: 0.4863 - val_loss: 1.0314 - val_accuracy: 0.5700 - 23s/epoch - 367ms/step
Epoch 2/15
16/16 [==============================] - 1s 42ms/step
63/63 - 21s - loss: 0.8570 - accuracy: 0.6528 - val_loss: 0.8850 - val_accuracy: 0.6300 - 21s/epoch - 334ms/step
Epoch 3/15
16/16 [==============================] - 1s 42ms/step
63/63 - 21s - loss: 0.7372 - accuracy: 0.7045 - val_loss: 0.7920 - val_accuracy: 0.7000 - 21s/epoch - 334ms/step
Epoch 4/15
16/16 [==============================] - 1s 42ms/step
63/63 - 21s - loss: 0.6711 - accuracy: 0.7348 - val_loss: 0.7970 - val_accuracy: 0.6600 - 21s/epoch - 335ms/step
Epoch 5/15
16/16 [==============================] - 1s 34ms/step
63/63 - 19s - loss: 0.5855 - accuracy: 0.7757 - val_loss: 0.6957 - val_accuracy: 0.7360 - 19s/epoch - 295ms/step
Epoch 6/15
16/16 [==============================

...

INFO:tensorflow:Assets written to: saved_model/ModelG 1/fit/run-10\assets


---Starting trial: run-11
{'filter_size': 7, 'filter_num': 96}
Epoch 1/15
16/16 [==============================] - 1s 72ms/step
63/63 - 31s - loss: 1.1503 - accuracy: 0.4910 - val_loss: 0.9976 - val_accuracy: 0.5860 - 31s/epoch - 488ms/step
Epoch 2/15
16/16 [==============================] - 1s 64ms/step
63/63 - 28s - loss: 0.8390 - accuracy: 0.6613 - val_loss: 0.8745 - val_accuracy: 0.6560 - 28s/epoch - 444ms/step
Epoch 3/15
16/16 [==============================] - 1s 63ms/step
63/63 - 27s - loss: 0.7265 - accuracy: 0.7045 - val_loss: 0.8460 - val_accuracy: 0.6280 - 27s/epoch - 428ms/step
Epoch 4/15
16/16 [==============================] - 1s 68ms/step
63/63 - 28s - loss: 0.6530 - accuracy: 0.7373 - val_loss: 0.7815 - val_accuracy: 0.6820 - 28s/epoch - 448ms/step
Epoch 5/15
16/16 [==============================] - 1s 62ms/step
63/63 - 28s - loss: 0.6017 - accuracy: 0.7615 - val_loss: 0.7274 - val_accuracy: 0.7380 - 28s/epoch - 446ms/step
Epoch 6/15
16/16 [=============================

INFO:tensorflow:Assets written to: saved_model/ModelG 1/fit/run-11\assets


---Starting trial: run-12
{'filter_size': 7, 'filter_num': 128}
Epoch 1/15
16/16 [==============================] - 1s 67ms/step
63/63 - 30s - loss: 1.2937 - accuracy: 0.4593 - val_loss: 1.1134 - val_accuracy: 0.5440 - 30s/epoch - 478ms/step
Epoch 2/15
16/16 [==============================] - 1s 64ms/step
63/63 - 27s - loss: 0.8691 - accuracy: 0.6528 - val_loss: 0.8739 - val_accuracy: 0.6580 - 27s/epoch - 432ms/step
Epoch 3/15
16/16 [==============================] - 1s 65ms/step
63/63 - 27s - loss: 0.7471 - accuracy: 0.7015 - val_loss: 0.8876 - val_accuracy: 0.6400 - 27s/epoch - 435ms/step
Epoch 4/15
16/16 [==============================] - 1s 63ms/step
63/63 - 27s - loss: 0.6602 - accuracy: 0.7385 - val_loss: 0.8433 - val_accuracy: 0.6500 - 27s/epoch - 422ms/step
Epoch 5/15
16/16 [==============================] - 1s 64ms/step
63/63 - 27s - loss: 0.6076 - accuracy: 0.7525 - val_loss: 0.7483 - val_accuracy: 0.7080 - 27s/epoch - 433ms/step
Epoch 6/15
16/16 [============================

INFO:tensorflow:Assets written to: saved_model/ModelG 1/fit/run-12\assets


In [7]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/ModelG 1/hparam_tuning'


In [8]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/ModelG 1/fit'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
